In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv(r"/content/drive/My Drive/Crypto/btcusd.csv")
df = df.groupby('time').mean()
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2841761 entries, 1364774820000 to 1593425100000
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   open    float64
 1   close   float64
 2   high    float64
 3   low     float64
 4   volume  float64
dtypes: float64(5)
memory usage: 130.1 MB


None

In [ ]:
#Features
df['High 8 RMA'] = df['close'].rolling(window=8).mean()
df['High 13 RMA'] = df['close'].rolling(window=13).mean()
df['High 21 RMA'] = df['close'].rolling(window=21).mean()
df['High 55 RMA'] = df['close'].rolling(window=55).mean()
df['Bollinger High'] = df['High 13 RMA'] +df['close'].rolling(2).std()
df['Bollinger Low'] =  df['High 13 RMA'] - df['close'].rolling(2).std()
df['returns'] = df['close'].pct_change(60)
df['8 EMA'] = df['close'].ewm(span=120,min_periods=0,adjust=False,ignore_na=False).mean()
df['13 EMA'] = df['close'].ewm(span=195,min_periods=0,adjust=False,ignore_na=False).mean()
df['21 EMA'] = df['close'].ewm(span=315,min_periods=0,adjust=False,ignore_na=False).mean()
df['34 EMA'] = df['close'].ewm(span=825,min_periods=0,adjust=False,ignore_na=False).mean()



#TARGETS
# # 1
# df['target']=(df['returns'].shift(-20)>0).astype(int)


# # 2
# # This might be something decent, but needs work
df['target'] = (((df['8 EMA']>df['13 EMA']) & (df['13 EMA']>df['21 EMA']) & (df['21 EMA']>df['34 EMA'])).astype(int).shift(-20)>0).astype(int)



#Drop all NA
df.dropna(inplace=True)

In [ ]:
def backtesting(prediction_column, percent_column):

  btc_price = 0
  one_week_df = backtesting_df[-43800:]
  potential_initial_investment = 1000
  potential_money_earned = 0
  potential_holdings = 0
  potential_status = 0
  actual_initial_investment = 1000
  actual_money_earned = 0
  actual_holdings = 0
  actual_status = 0
  # 0 Statusmeans we are not holding money. If status is 1, we are invested
#Ideal
  for i in range(len(one_week_df)):
    if (one_week_df.iloc[i]['target']==1 and potential_status ==0):
      amount_invested = potential_initial_investment*.3
      potential_holdings = (amount_invested)/(one_week_df.iloc[i]['close'])
      potential_status = 1
      potential_initial_investment = potential_initial_investment-amount_invested

    elif (one_week_df.iloc[i]['target']==0 and potential_status ==1):
      potential_money_earned = potential_holdings*one_week_df.iloc[i]['close']
      potential_holdings = 0
      potential_initial_investment += potential_money_earned
      potential_status = 0
    else:
      pass

#Model
  for i in range(len(one_week_df)):

#Stop loss attempt
    if (btc_price != 0 and actual_status == 1 and (btc_price*.98)>=one_week_df.iloc[i]['open']):
      actual_money_earned = actual_holdings*one_week_df.iloc[i]['close']
      actual_initial_investment += actual_money_earned
      actual_holdings = 0
      actual_status = 0
      btc_price = 0
#Buy
    if (one_week_df.iloc[i][prediction_column]==1 and actual_status == 0 and one_week_df.iloc[i][percent_column]>0.52):
      amount_invested = actual_initial_investment*.3
      actual_holdings = (amount_invested)/(one_week_df.iloc[i]['close'])
      actual_status = 1
      actual_initial_investment = actual_initial_investment-amount_invested
      btc_price = one_week_df.iloc[i]['close']
#Sell
    elif (one_week_df.iloc[i][prediction_column]==0 and actual_status == 1 and one_week_df.iloc[i][percent_column]>0.65):
      actual_money_earned = actual_holdings*one_week_df.iloc[i]['close']
      actual_initial_investment += actual_money_earned
      actual_holdings = 0
      actual_status = 0
      btc_price = 0
    else:
      pass

  print(f"strategy optimally earns: ${potential_initial_investment} in one weeks")
  print(f"{prediction_column} strategy earns: ${actual_initial_investment} in one week")


# Dual model prediction
def dual_backtesting(buy_prediction_column, buy_percent_column, sell_prediction_column, sell_percent_column):

  btc_price = 0
  one_week_df = backtesting_df[-43800:]
  potential_initial_investment = 1000
  potential_money_earned = 0
  potential_holdings = 0
  potential_status = 0
  actual_initial_investment = 1000
  actual_money_earned = 0
  actual_holdings = 0
  actual_status = 0
  # 0 Status means we are not holding money. If status is 1, we are invested



# Ideal
  for i in range(len(one_week_df)):
    if (one_week_df.iloc[i]['target']==1 and potential_status ==0):
      amount_invested = potential_initial_investment*.3
      potential_holdings = (amount_invested)/(one_week_df.iloc[i]['close'])
      potential_status = 1
      potential_initial_investment = potential_initial_investment-amount_invested

    elif (one_week_df.iloc[i]['target']==0 and potential_status ==1):
      potential_money_earned = potential_holdings*one_week_df.iloc[i]['close']
      potential_holdings = 0
      potential_initial_investment += potential_money_earned
      potential_status = 0
    else:
      pass

# Models
  for i in range(len(one_week_df)):
    #Stop loss attempt
    if (btc_price != 0 and actual_status == 1 and (btc_price*.98)>=one_week_df.iloc[i]['open']):
      actual_money_earned = actual_holdings*one_week_df.iloc[i]['close']
      actual_initial_investment += actual_money_earned
      actual_holdings = 0
      actual_status = 0
      btc_price = 0

    # Buy
    if (one_week_df.iloc[i][buy_prediction_column]==1 and actual_status == 0 and one_week_df.iloc[i][buy_percent_column]>=0.55):
      amount_invested = actual_initial_investment*.3
      btc_price = one_week_df.iloc[i]['close']
      actual_holdings = (amount_invested)/(one_week_df.iloc[i]['close'])
      actual_status = 1
      actual_initial_investment = actual_initial_investment-amount_invested
    #Sell 
    elif (one_week_df.iloc[i][sell_prediction_column]==0 and actual_status == 1 and one_week_df.iloc[i][sell_percent_column]>=0.5):
      actual_money_earned = actual_holdings*one_week_df.iloc[i]['close']
      actual_initial_investment += actual_money_earned
      actual_holdings = 0
      actual_status = 0
      btc_price = 0
    else:
      pass

  print(f"strategy optimally earns: ${potential_initial_investment} in one weeks")
  print(f"{buy_prediction_column} and {sell_prediction_column} strategy earns: ${actual_initial_investment} in one week") 

103.9

In [ ]:
scaler = MinMaxScaler()
x = df.drop(['returns', 'target'], axis = 1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 69, shuffle = False)
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
backtesting_df = df[-568305:].copy()

In [ ]:
len(LGR_prob)

568305

In [ ]:
#LOG
LGR = LogisticRegression(class_weight = 'balanced', random_state = 69)
LGR.fit(X_train, y_train)
predictions = LGR.predict_proba(X_test)

#Format for DF and backtesting
LGR_prob = []
LGR_perc = []
for i in range(len(predictions)):
  if predictions[i][0]>predictions[i][1]:
    LGR_prob.append(0)
    LGR_perc.append(predictions[i][0])
  else:
    LGR_prob.append(1)
    LGR_perc.append(predictions[i][1])

backtesting_df['Log_predictions'] = LGR_prob
backtesting_df['Log_percent'] = LGR_perc


In [ ]:
#GBC

XGB_predict = []
XGB_prob = []
XGBC = XGBClassifier(max_depth = 6, learning_rate=.3, random_state=69)
XGBC.fit(X_train, y_train)
predictions = XGBC.predict_proba(X_test)

for i in range(len(predictions)):
  if predictions[i][0]>predictions[i][1]:
    XGB_predict.append(0)
    XGB_prob.append(predictions[i][0])
  else:
    XGB_predict.append(1)
    XGB_prob.append(predictions[i][1])
backtesting_df['XGB_predict'] = XGB_predict
backtesting_df['XGB_prob'] = XGB_prob

In [ ]:
# $985.56 no stoploss 52 buy
backtesting("Log_predictions", "Log_percent")

strategy optimally earns: $1054.243353039738 in one weeks
Log_predictions strategy earns: $993.1753263008405 in one week


In [ ]:
# $1008 no stoploss 52 buy
# $1009 stoploss
backtesting("XGB_predict", "XGB_prob")

strategy optimally earns: $1054.243353039738 in one weeks
XGB_predict strategy earns: $1009.0226677708276 in one week


In [ ]:
backtesting_df.columns

Index(['open', 'close', 'high', 'low', 'volume', 'High 8 RMA', 'High 13 RMA',
       'High 21 RMA', 'High 55 RMA', 'Bollinger High', 'Bollinger Low',
       'returns', '8 EMA', '13 EMA', '21 EMA', '34 EMA', 'target',
       'predictions', 'percent', 'XGB_predict', 'XGB_prob'],
      dtype='object')

In [ ]:
# Highest XLLL simple 20 min no stoploss:  $1008.21
# XLXX 1009/ ideal 1054 
# Log_percent, Log_predictions
# XGB_prob, XGB_predict
dual_backtesting("Log_predictions", "XGB_prob", "XGB_predict", "XGB_prob" )

strategy optimally earns: $1054.243353039738 in one weeks
Log_predictions and XGB_predict strategy earns: $980.9407603031376 in one week
